In [ ]:
from VR_Trajectory_analysis import *

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20250217_GreenBlue_Geometry_Data/RunData'
df1 = get_combined_df(directory, trim_seconds=1.0)
directory = '/Users/apaula/ownCloud/MatrexVR1/20250226_BlueGreenBlue_Geometry_Data/RunData'
df2 = get_combined_df(directory, trim_seconds=1.0)
directory = '/Users/apaula/ownCloud/MatrexVR1/20250121_ants_StartingPosition_Data/RunData'
df3 = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
df_combined = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
df_combined["FlyID"].nunique()


In [ ]:
df = df_combined.copy()

In [ ]:
df = add_trial_id_and_displacement(df)
df = add_trial_time(df)

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_displacement(df[df['Scene']=='Choice_noBG'], min_disp=0, max_disp=500)

In [ ]:
#plot_trajectories(df_normal, 'normal')

In [ ]:
df_3cyl = df_normal[df_normal['ConfigFile'] == '3Cylinders111_constantSize.json'].copy()


In [ ]:
def get_first_goal_reached(df_normal,
                           goals,
                           threshold=3.5):
    """
    Given a dataframe of trial data, determine the first goal reached 
    and the time at which it was reached for each UniqueTrialID.
    """
    
    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    results = []
    
    # Group by UniqueTrialID
    for trial_id, trial_data in df_normal.groupby('UniqueTrialID'):
        config = trial_data['ConfigFile'].iloc[0]
        
        # Sort by time
        trial_data = trial_data.sort_values(by='trial_time')
        
        first_reached = None
        reached_time = None
        
        for idx, row in trial_data.iterrows():
            participant_pos = (row['GameObjectPosX'], row['GameObjectPosZ'])
            
            # Check each goal
            for goal_name, goal_pos in goals:
                if distance(participant_pos, goal_pos) <= threshold:
                    first_reached = goal_name
                    reached_time = row['trial_time']
                    break
            
            if first_reached is not None:
                break
        
        results.append((trial_id, config, first_reached, reached_time))
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results, columns=[
        'UniqueTrialID',
        'ConfigFile',
        'FirstReachedGoal',
        'GoalReachedTime'
    ])
    
    return results_df

In [ ]:
# 3. Run first goal reached for triple-goal
triple_left_goal = (-20.5212, 56.381557)
triple_center_goal = (0, 60)
triple_right_goal = (20.5212, 56.381557)

results_3cyl = get_first_goal_reached(
    df_3cyl,
    goals=[
        ('triple_left', triple_left_goal),
        ('triple_center', triple_center_goal),
        ('triple_right', triple_right_goal)
    ],
    threshold=3.5
)


In [ ]:
# 1. Keep only trials that actually reached a goal
valid_results = results_3cyl.dropna(subset=['FirstReachedGoal'])

# 2. Merge the cutoff times back into df
#    We merge on 'UniqueTrialID' to get each trial's GoalReachedTime.
df_merged = pd.merge(df, valid_results[['UniqueTrialID', 'GoalReachedTime', 'FirstReachedGoal']], on='UniqueTrialID', how='inner')

# 3. Filter df so that only rows with trial_time less than or equal to the goal time are kept
df_cut = df_merged[df_merged['trial_time'] <= df_merged['GoalReachedTime']]

In [ ]:
# Save DataFrames as pickle (binary, preserves dtypes & indexes)
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/3goals/df_cut.pkl'
df_cut.to_pickle(df_cut_path_pickle)

In [ ]:
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/3goals/df_cut.pkl'
df_cut = pd.read_pickle(df_cut_path_pickle)

In [ ]:
df_cut["FlyID"].nunique()

In [ ]:
# 4. Plot the trajectories and show how many trials reached each goal
fig, ax = plt.subplots(figsize=(8, 6))

# Plot each trial’s trajectory
for trial_id, trial_data in df_cut.groupby('UniqueTrialID'):
    ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)

# Basic plot settings
ax.set_ylim(20, 60)
ax.set_xlim(-20, 20)
ax.set_aspect('equal', adjustable='box')
ax.set_title("Trajectories: 3Cylinders111_constantSize.json")
ax.set_xlabel("X Position (cm)")
ax.set_ylabel("Z Position (cm)")
ax.grid(True)

# Count how many reached each of the three possible goals
left_count = (valid_results['FirstReachedGoal'] == 'triple_left').sum()
center_count = (valid_results['FirstReachedGoal'] == 'triple_center').sum()
right_count = (valid_results['FirstReachedGoal'] == 'triple_right').sum()

total = left_count + center_count + right_count
if total > 0:
    left_ratio = left_count / total
    center_ratio = center_count / total
    right_ratio = right_count / total
    ratio_text = (f"Left: {left_count} ({left_ratio:.2f}), "
                  f"Center: {center_count} ({center_ratio:.2f}), "
                  f"Right: {right_count} ({right_ratio:.2f})")
else:
    ratio_text = "No goals reached"

# Add text box with ratio information
ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
        bbox=dict(boxstyle="round", fc="w", ec="0.5"))

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assume df_cut is your DataFrame of interest.

# 1. Create a mirrored copy of df_cut
df_mirrored = df_cut.copy()

# 2. Modify the x-coordinate by multiplying by -1
df_mirrored['GameObjectPosX'] = -df_mirrored['GameObjectPosX']

# 3. (Optional) Modify the UniqueTrialID to mark these as mirrored duplicates
#    so you can tell them apart in groupby plots, etc.
df_mirrored['UniqueTrialID'] = df_mirrored['UniqueTrialID'].astype(str) + "_mirror"

# 4. Concatenate original and mirrored data
df_duplicated = pd.concat([df_cut, df_mirrored], ignore_index=True)

# 5. Now df_duplicated has both the original and mirrored trajectories.
#    Plot them together or separately as needed.

fig, ax = plt.subplots(figsize=(8,6))

for trial_id, trial_data in df_duplicated.groupby('UniqueTrialID'):
    ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.2)
    
ax.set_ylim(20, 60)
ax.set_xlim(-20, 20)
ax.set_aspect('equal', adjustable='box')
ax.set_title("Original + Mirrored Trajectories")
ax.set_xlabel("X Position (cm)")
ax.set_ylabel("Z Position (cm)")
ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.cm as cm
from matplotlib.collections import LineCollection

fig, ax = plt.subplots(figsize=(8, 6))

# ---------- 1. Tell matplotlib how to translate trial_time → colour ----------
norm = plt.Normalize(df_duplicated['trial_time'].min(),
                     df_duplicated['trial_time'].max())  # global scale
cmap = cm.viridis                                     # pick your favourite map

# ---------- 2. Plot every trial as a coloured LineCollection ----------
for trial_id, trial_data in df_duplicated.groupby("UniqueTrialID"):
    x = trial_data['GameObjectPosX'].values
    z = trial_data['GameObjectPosZ'].values
    t = trial_data['trial_time'].values               # colour driver

    # build segments:  [[(x0,z0),(x1,z1)], [(x1,z1),(x2,z2)], …]
    pts       = np.column_stack([x, z]).reshape(-1, 1, 2)
    segments  = np.concatenate([pts[:-1], pts[1:]], axis=1)

    lc = LineCollection(segments,
                        cmap=cmap,
                        norm=norm,
                        array=t[:-1],      # one colour per segment
                        linewidth=2,
                        alpha=0.2)
    ax.add_collection(lc)

# ---------- 3. Usual formatting ----------
ax.set_ylim(20, 60)
ax.set_xlim(-20, 20)
ax.set_aspect('equal', adjustable='box')
ax.set_title("Original + Mirrored Trajectories\n(coloured by trial time)")
ax.set_xlabel("X Position (cm)")
ax.set_ylabel("Z Position (cm)")
ax.grid(False)

# ---------- 4. Colour-bar for reference ----------
cb = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),
                  ax=ax, label="Trial time (s)")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assume df_duplicated is the DataFrame containing (X, Z) for both original and mirrored trajectories

fig, ax = plt.subplots(figsize=(6, 5))

# Plot a 2D histogram
# 'bins' can be a single integer or a tuple/list specifying bins for X and Z
# 'range' sets the min/max for X and Z. Adjust to suit your data.
h = ax.hist2d(
    df_duplicated['GameObjectPosX'], 
    df_duplicated['GameObjectPosZ'], 
    bins=(32, 32),                 # 40 bins along each dimension (adjust as needed)
    range=[[-20, 20], [20, 60]],   # X range: -20 to 20, Z range: 20 to 60
    cmap='viridis'
)

# Add a colorbar to show counts/density
fig.colorbar(h[3], ax=ax, label='Count')

ax.set_xlabel("X Position (cm)")
ax.set_ylabel("Z Position (cm)")
ax.set_title("2D Histogram of (X, Z) Positions (Original + Mirrored)")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------------
# 0)  PARAMETERS — tweak these as you like
n_xbins  = 64              # spatial resolution in X
n_zbins  = 64              # spatial resolution in Z
n_tbins  = 64              # how many time buckets to break trial_time into

x_range  = (-20, 20)       # cm   (X axis)
z_range  = (20,  60)       # cm   (Z axis)

# ------------------------------------------------------------------
# 1)  BASIC ARRAYS
x = df_duplicated['GameObjectPosX'].to_numpy()
z = df_duplicated['GameObjectPosZ'].to_numpy()
t = df_duplicated['trial_time'   ].to_numpy()

# common bin edges for every histogram
x_edges = np.linspace(*x_range, n_xbins + 1)
z_edges = np.linspace(*z_range, n_zbins + 1)
t_edges = np.linspace(t.min(), t.max(), n_tbins + 1)  # equal-width slices

# ------------------------------------------------------------------
# 2)  BUILD ONE HISTOGRAM PER TIME-BIN
H_slices = []               # list of 2-D arrays  (shape each: [n_xbins, n_zbins])
for i in range(n_tbins):
    mask = (t >= t_edges[i]) & (t < t_edges[i+1])
    H, *_ = np.histogram2d(x[mask], z[mask],
                           bins=(x_edges, z_edges))
    H_slices.append(H)      # keep raw counts  (no normalisation yet)

# ------------------------------------------------------------------
# 3)  FIGURE 1 — SMALL MULTIPLES OF ALL SLICES
n_cols = int(np.ceil(np.sqrt(n_tbins)))
n_rows = int(np.ceil(n_tbins / n_cols))
fig1, axes = plt.subplots(n_rows, n_cols,
                          figsize=(3.5 * n_cols, 3.5 * n_rows),
                          constrained_layout=True)

vmax_global = np.max(H_slices)   # common colour scale so intensities are comparable

for idx, (ax, H_i) in enumerate(zip(axes.flat, H_slices)):
    im = ax.imshow(H_i.T,               # transpose so rows = Z
                   origin='lower',
                   extent=[x_edges[0], x_edges[-1], z_edges[0], z_edges[-1]],
                   aspect='equal',
                   cmap='viridis',
                   vmin=0, vmax=vmax_global)
    ax.set_title(f"{t_edges[idx]:.1f} – {t_edges[idx+1]:.1f} s")
    ax.set_xlabel("X (cm)"); ax.set_ylabel("Z (cm)")

# hide any empty panels
for ax in axes.flat[len(H_slices):]:
    ax.axis("off")

cax = fig1.add_axes([0.92, 0.15, 0.02, 0.7])       # colour-bar
fig1.colorbar(im, cax=cax, label="Count in bin")

# ------------------------------------------------------------------
# 4)  FIGURE 2 — ADDITIVE COMPOSITE (simple sum)
H_sum = np.sum(H_slices, axis=0)                   # shape (n_xbins, n_zbins)

fig2, ax2 = plt.subplots(figsize=(6, 5))
csum = ax2.imshow(H_sum.T,
                  origin='lower',
                  extent=[x_edges[0], x_edges[-1], z_edges[0], z_edges[-1]],
                  aspect='equal',
                  cmap='magma',
                  vmin=0, vmax=600)
ax2.set_xlabel("X (cm)"); ax2.set_ylabel("Z (cm)")
ax2.set_title("Additive composite of all time slices")
fig2.colorbar(csum, ax=ax2, label="Σ (counts)")

plt.show()
# ---- already have H_sum from previous step ----
H_sq = H_sum ** 2                                  # element-wise squaring

# optional: rescale if you want the colour scale comparable
# H_sq = H_sq / H_sq.max() * H_sum.max()

# ---- Figure 3 — squared composite ---------------------------------
fig3, ax3 = plt.subplots(figsize=(6, 5))
csq = ax3.imshow(H_sq.T,
                 origin='lower',
                 extent=[x_edges[0], x_edges[-1], z_edges[0], z_edges[-1]],
                 aspect='equal',
                 cmap='magma',
                 vmin=0, vmax=200000   )
ax3.set_xlabel("X (cm)"); ax3.set_ylabel("Z (cm)")
ax3.set_title("Additive composite — squared (highlights hotspots)")
fig3.colorbar(csq, ax=ax3, label="(Σ counts)²")
plt.show()

# ------------------------------------------------------------
# 5)  FIGURE 4 — Gaussian-blurred (σ in "histogram pixels")
# ------------------------------------------------------------
from scipy.ndimage import gaussian_filter        # pip install scipy

sigma_px = 1.2                                  # try 1–3; larger ⇒ smoother
H_blur   = gaussian_filter(H_sum, sigma=sigma_px, mode='nearest')

fig4, ax4 = plt.subplots(figsize=(6, 5))
cblur = ax4.imshow(
    H_blur.T,
    origin='lower',
    extent=[x_edges[0], x_edges[-1], z_edges[0], z_edges[-1]],
    aspect='equal',
    cmap='magma',
    vmin=0, vmax=600           # adjust to taste (or use e.g. np.percentile)
)

ax4.set_xlabel("X (cm)")
ax4.set_ylabel("Z (cm)")
ax4.set_title(f"Squared composite — Gaussian blur  (σ = {sigma_px})")
fig4.colorbar(cblur, ax=ax4, label="blurred (Σ counts)²")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --------------------------------------------------------------------
# 1) Create or load your data
# Suppose 'df_duplicated' holds both original and mirrored data,
# with columns 'GameObjectPosX' (X) and 'GameObjectPosZ' (Z).

x_data = df_duplicated['GameObjectPosX'].values
z_data = df_duplicated['GameObjectPosZ'].values

# --------------------------------------------------------------------
# 2) Build a 2D histogram of size (n_zbins x n_xbins)
#    We'll use np.histogram2d(x, z) such that:
#       H[i, j] = count of points in x_bin_i, z_bin_j
#    By default: i -> X-bins, j -> Z-bins.
#    We'll flip that below for plotting so that Z is "vertical".

n_xbins = 64
n_zbins = 64

x_range = (-20, 20)
z_range = (20, 60)

H, xedges, zedges = np.histogram2d(
    x_data, 
    z_data, 
    bins=(n_xbins, n_zbins),
    range=[x_range, z_range]
)

# H.shape == (n_xbins, n_zbins)
#  - The first axis is X-bin index (rows in H)
#  - The second axis is Z-bin index (columns in H)

# --------------------------------------------------------------------
# 3) Transpose H so that:
#      H_plot[z_bin, x_bin]
#    Because we want H_plot to have shape (n_zbins, n_xbins),
#    so that z_bin is the row index and x_bin is the column index.
#    Then we can display with imshow or pcolormesh 
#    so that the vertical axis corresponds to Z.
H_plot = H.T  # shape: (n_zbins, n_xbins)

# --------------------------------------------------------------------
# 4) Per-row (per-Z-bin) min–max scaling
#    For each row z_idx, find the min and max in that row.
#    Then normalize so that row values go from 0 to 1.
for z_idx in range(H_plot.shape[0]):
    row_min = H_plot[z_idx, :].min()
    row_max = H_plot[z_idx, :].max()
    if row_max > row_min:
        H_plot[z_idx, :] = (H_plot[z_idx, :] - row_min) / (row_max - row_min)
    else:
        # If the entire row is zero (no data), do nothing or keep it zero
        pass

# After this loop, each row has values in [0,1].

# --------------------------------------------------------------------
# 5) Plot using imshow (or pcolormesh)
#    We'll use imshow for simplicity. We set origin='lower'
#    so that the first row of the array is at the bottom of the plot
#    (smallest Z). We'll also provide 'extent' so the axes match
#    the actual X and Z ranges.

fig, ax = plt.subplots(figsize=(8,6))

# imshow expects array shape (nrows, ncols), which we have as (n_zbins, n_xbins)
img = ax.imshow(
    H_plot, 
    origin='lower', 
    aspect='auto',
    cmap='viridis',
    extent=[xedges[0], xedges[-1], zedges[0], zedges[-1]]
)

ax.set_xlabel("X Position (cm)")
ax.set_ylabel("Z Position (cm)")
ax.set_title("2D Per-Z-bin Normalized Heat Map")

# The colorbar now indicates the normalized range [0,1] 
# with 1.0 meaning "highest count in that specific row (Z-bin)."
cbar = fig.colorbar(img, ax=ax, label="Normalized count per Z-bin")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# --------------------------------------------------------------------
# 1) Create or load your data
# Suppose 'df_duplicated' holds both original and mirrored data,
# with columns 'GameObjectPosX' (X) and 'GameObjectPosZ' (Z).

x_data = df_duplicated['GameObjectPosX'].values
z_data = df_duplicated['GameObjectPosZ'].values

# --------------------------------------------------------------------
# 2) Build a 2D histogram of size (n_zbins x n_xbins)
n_xbins = 64
n_zbins = 64

x_range = (-20, 20)
z_range = (20, 60)

H, xedges, zedges = np.histogram2d(
    x_data, 
    z_data, 
    bins=(n_xbins, n_zbins),
    range=[x_range, z_range]
)

# --------------------------------------------------------------------
# 3) Transpose H for plotting so that Z is the vertical axis
H_plot = H.T  # shape: (n_zbins, n_xbins)

# --------------------------------------------------------------------
# 4) Per-row (per-Z-bin) min–max scaling
for z_idx in range(H_plot.shape[0]):
    row_min = H_plot[z_idx, :].min()
    row_max = H_plot[z_idx, :].max()
    if row_max > row_min:
        H_plot[z_idx, :] = (H_plot[z_idx, :] - row_min) / (row_max - row_min)

# --------------------------------------------------------------------
# 5) Apply a Gaussian blur
#    The 'sigma' parameter controls how blurry the result is.
#    You can experiment with different sigma values (e.g. 0.5, 1, 2, ...)
#    The mode='nearest' helps with edges by replicating the edge values.
H_blurred = gaussian_filter(H_plot, sigma=1.0, mode='nearest')

# --------------------------------------------------------------------
# 6) Plot using imshow
fig, ax = plt.subplots(figsize=(8,6))

img = ax.imshow(
    H_blurred,      # <-- Display the blurred data
    origin='lower', 
    aspect='auto',
    cmap='viridis',
    extent=[xedges[0], xedges[-1], zedges[0], zedges[-1]]
)

ax.set_xlabel("X Position (cm)")
ax.set_ylabel("Z Position (cm)")
ax.set_title("2D Per-Z-bin Normalized Heat Map (Blurred)")

cbar = fig.colorbar(img, ax=ax, label="Normalized count per Z-bin (blurred)")

plt.tight_layout()
plt.show()
